In [1]:
!pip install jieba

     ---------------------------------------- 19.2/19.2 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314459 sha256=71faa854401081780e25d3e0e8ef9f29b2b53b8b72688ee37ee01574a435b682
  Stored in directory: c:\users\moutsea\appdata\local\pip\cache\wheels\7d\74\cf\08c94db4b784e2c1ef675a600b7b5b281fd25240dcb954ee7e
Successfully built jieba


In [2]:
import jieba

In [3]:
import re

def remove_punctuation(text):
    # 定义中英文常见标点符号
    punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~。，、；：？！…—～·《》「」『』（）〔〕【】〈〉"""
    
    # 使用 re.sub 函数替换掉文本中的标点符号
    text_no_punctuation = re.sub(f"[{re.escape(punctuation)}]", "", text)
    
    return text_no_punctuation

# 示例文本，包含中英文标点符号
text = "Hello, world! 你好，世界！"

# 移除标点符号
text_no_punctuation = remove_punctuation(text)

print("原始文本:", text)
print("移除标点后:", text_no_punctuation)


原始文本: Hello, world! 你好，世界！
移除标点后: Hello world 你好世界


In [25]:
from collections import Counter

good_file = './data/jd_comment/good.txt'
bad_file = './data/jd_comment/bad.txt'

def prepare_data(good_file, bad_file, filter=True):
    all_words, pos_sentences, neg_sentences = [], [], []

    # 定义处理每一行的内部函数
    def process_line(line, sentence_list):
        if filter:
            line = remove_punctuation(line)
        words = jieba.lcut(line.strip())
        if words:
            all_words.extend(words)
            sentence_list.append(words)

    def process_file(file_path, sentence_list):
        with open(file_path, 'r') as f:
            for line in f:
                process_line(line, sentence_list)

    process_file(good_file, pos_sentences)
    process_file(bad_file, neg_sentences)

    # 使用列表推导式和enumerate生成词典
    dit = {word: [idx, freq] for idx, (word, freq) in enumerate(Counter(all_words).items())}

    return pos_sentences, neg_sentences, dit

In [26]:
pos_sentences, neg_sentences, dit = prepare_data(good_file, bad_file)

In [27]:
st = sorted([(v[1], w) for w, v in dit.items()])

In [28]:
def word2index(word, dit):
    if word in dit:
        return dit[word][0]
    return -1

def index2word(idx, dit):
    if idx < len(dit):
        return list(dit.items())[idx][0]
    return None

In [29]:
word2index('好', dit)

19

In [30]:
index2word(19, dit)

'好'

In [37]:
import numpy as np
dataset, labels = [], []

def sentence_to_sample(samples, label, dit):

    def sentence_to_vec(data, dit):
        vector = np.zeros(len(dit))
        for w in data:
            vector[w] += 1
        # 归一化
        return (1.0 * vector / len(data))
    
    for sample in samples:
        data = []
        for w in sample:
            if w in dit:
                data.append(word2index(w, dit))
        dataset.append(sentence_to_vec(data, dit))
        labels.append(label)

In [38]:
sentence_to_sample(pos_sentences, 0, dit)
sentence_to_sample(neg_sentences, 1, dit)